In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mserr
from sklearn.decomposition import PCA

In [55]:
path='C:\\Users\\personal\\Desktop\\MSIT\\IT\\DataScience_2019501026\\Intro to ML\\code camp\\house-prices-advanced-regression-techniques\\'
train = path + "train.csv"
test = path + "test.csv"

In [56]:
trainx_df = pd.read_csv(train, index_col = 'Id')
print(trainx_df.shape)

(1460, 80)


In [57]:
trainy_df = trainx_df['SalePrice']
print(trainy_df.shape)

(1460,)


In [58]:
trainx_df.drop('SalePrice', axis=1, inplace=True)
testx_df = pd.read_csv(test, index_col='Id')
print(trainx_df.isnull().sum())

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 79, dtype: int64


In [59]:
sample_size = len(trainx_df)
columns_with_null_values = []
columns_with_null_values = [[col, float(trainx_df[col].isnull().sum())/float(sample_size)] for col in trainx_df.columns if trainx_df[col].isnull().sum()]
print(columns_with_null_values)

[['LotFrontage', 0.1773972602739726], ['Alley', 0.9376712328767123], ['MasVnrType', 0.005479452054794521], ['MasVnrArea', 0.005479452054794521], ['BsmtQual', 0.025342465753424658], ['BsmtCond', 0.025342465753424658], ['BsmtExposure', 0.026027397260273973], ['BsmtFinType1', 0.025342465753424658], ['BsmtFinType2', 0.026027397260273973], ['Electrical', 0.0006849315068493151], ['FireplaceQu', 0.4726027397260274], ['GarageType', 0.05547945205479452], ['GarageYrBlt', 0.05547945205479452], ['GarageFinish', 0.05547945205479452], ['GarageQual', 0.05547945205479452], ['GarageCond', 0.05547945205479452], ['PoolQC', 0.9952054794520548], ['Fence', 0.8075342465753425], ['MiscFeature', 0.963013698630137]]


In [60]:
columns_to_drop = [x for (x,y) in columns_with_null_values if y>.3]
print(columns_to_drop)

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


In [61]:
trainx_df.drop(columns_to_drop, axis = 1, inplace=True)
testx_df.drop(columns_to_drop, axis=1, inplace=True)

In [62]:
categorical_columns = [col for col in trainx_df.columns if
                      trainx_df[col].dtype == object]
ordinal_columns = [col for col in trainx_df.columns if col not in categorical_columns]
# categorical_columns.append('MSSubClass')
print(categorical_columns)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


In [63]:
dummy_row = list()
for col in trainx_df.columns:
    if col in categorical_columns:
        dummy_row.append("dummy")
    else:
        dummy_row.append("")
print(dummy_row)

['', 'dummy', '', '', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', '', '', '', '', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', '', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', 'dummy', '', 'dummy', '', '', '', 'dummy', 'dummy', 'dummy', 'dummy', '', '', '', '', '', '', '', '', '', '', 'dummy', '', 'dummy', '', 'dummy', '', 'dummy', '', '', 'dummy', 'dummy', 'dummy', '', '', '', '', '', '', '', '', '', 'dummy', 'dummy']


In [64]:
new_row = pd.DataFrame([dummy_row], columns = trainx_df.columns)
trainx_df = pd.concat([trainx_df, new_row], axis = 0, ignore_index = True)
testx_df = pd.concat([testx_df], axis = 0, ignore_index = True)
# trainx_df.to_csv["dummy.csv"]
for col in categorical_columns:
    trainx_df[col].fillna(value = "dummy", inplace = True)
    testx_df[col].fillna(value = "dummy", inplace = True)

In [65]:
enc = OneHotEncoder(drop='first',sparse=False)
enc.fit(trainx_df[categorical_columns])
#print(enc.get_feature_names(categorical_columns))
trainx_enc=pd.DataFrame(enc.transform(trainx_df[categorical_columns]))
testx_enc = pd.DataFrame(enc.transform(testx_df[categorical_columns]))
trainx_enc.columns=enc.get_feature_names(categorical_columns)
testx_enc.columns=enc.get_feature_names(categorical_columns)
trainx_df=pd.concat([trainx_df[ordinal_columns],trainx_enc],axis=1,ignore_index=
                    True)
testx_df=pd.concat([testx_df[ordinal_columns],testx_enc],axis=1,ignore_index=
                    True)
trainx_df.drop(trainx_df.tail(1).index,inplace=True)
imputer = KNNImputer(n_neighbors=2)
imputer.fit(trainx_df)
trainx_df_filled=imputer.transform(trainx_df)
trainx_df_filled = pd.DataFrame(trainx_df_filled,columns=trainx_df.columns)
testx_df_filled=imputer.transform(testx_df)
testx_df_filled = pd.DataFrame(testx_df_filled,columns=testx_df.columns)
testx_df_filled.reset_index(drop=True,inplace=True)
print(trainx_df_filled.isnull().sum())

0      0
1      0
2      0
3      0
4      0
      ..
265    0
266    0
267    0
268    0
269    0
Length: 270, dtype: int64


In [68]:
scale = 'Standard'
if scale == 'Standard':
    scaler = preprocessing.StandardScaler().fit(trainx_df)
    trainx_df=scaler.transform(trainx_df)
    testx_df=scaler.transform(testx_df)
elif scale == 'MinMax':
    scaler=preprocessing.MinMaxScaler().fit(trainx_df)
    trainx_df=scaler.transform(trainx_df)
    testx_df=scaler.transform(testx_df)
print(trainx_df,testx_df)

[[ 0.07337496 -0.20803433 -0.20714171 ...  0.4676514  -0.30599503
   0.        ]
 [-0.87256276  0.40989452 -0.09188637 ...  0.4676514  -0.30599503
   0.        ]
 [ 0.07337496 -0.08444856  0.07347998 ...  0.4676514  -0.30599503
   0.        ]
 ...
 [ 0.30985939 -0.16683907 -0.14781027 ...  0.4676514  -0.30599503
   0.        ]
 [-0.87256276 -0.08444856 -0.08016039 ...  0.4676514  -0.30599503
   0.        ]
 [-0.87256276  0.20391824 -0.05811155 ...  0.4676514  -0.30599503
   0.        ]] [[-0.87256276  0.40989452  0.11076257 ...  0.4676514  -0.30599503
   0.        ]
 [-0.87256276  0.45108977  0.37584985 ...  0.4676514  -0.30599503
   0.        ]
 [ 0.07337496  0.16272298  0.33205282 ...  0.4676514  -0.30599503
   0.        ]
 ...
 [-0.87256276  3.70551504  0.95042275 ... -2.13834494 -0.30599503
   0.        ]
 [ 0.66458604 -0.3316201  -0.00759964 ...  0.4676514  -0.30599503
   0.        ]
 [ 0.07337496  0.16272298 -0.08918038 ...  0.4676514  -0.30599503
   0.        ]]
